In [1]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline

from mne.decoding import CSP
from mne.channels import read_layout
from loaddata import *
from sklearn import metrics as met

print(__doc__)

Automatically created module for IPython interactive environment


In [2]:
fp = {
    'al': './data_set_raw/data_set_IVa_al.mat',
}

pick_chan = {
    'al': ['C3', 'Cz', 'C5'],
}

low_freq, high_freq = 7., 30.
tmin, tmax = 0., 3.5

# event_id
event_id = {'right': 1, 'foot': 2}

In [3]:
#Se carga set de datos crudos
raw = creatRawArray(fp['al'])

# Se aplica filtros band-pass
raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')

#Se carga eventos
events, labels = creatEventsArray(fp['al'])

#Se genera las epocas con los datos crudos y los eventos
epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                    verbose=False)

epochs_train = epochs.copy().crop(tmin=1., tmax=2.)

#convierte 1 -> -1 y 2 -> 0
labels = epochs.events[:, -1] - 2

#Lo convierte a matriz numpy
epochs_data = epochs_train.get_data()


train_x=epochs_data[:224]
train_y=labels[:224]

test_x =epochs_data[224:]
test_y=labels[224:]


Creating RawArray with float64 data, n_channels=118, n_times=283574
    Range : 0 ... 283573 =      0.000 ...  2835.730 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 165 samples (1.650 sec)



In [4]:
# Assemble a classifier
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

# Use scikit-learn Pipeline
clf = Pipeline([('CSP', csp), ('LDA', lda)])


clf.fit(train_x, train_y)
result=clf.predict(test_x)

print(result)
print(test_y)

Computing rank from data with rank=None
    Using tolerance 2e+02 (2.2e-16 eps * 118 dim * 7.7e+15  max singular value)
    Estimated rank (mag): 118
    MAG: rank 118 computed from 118 data channels with 0 projectors
Reducing data rank from 118 -> 118
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2e+02 (2.2e-16 eps * 118 dim * 7.8e+15  max singular value)
    Estimated rank (mag): 118
    MAG: rank 118 computed from 118 data channels with 0 projectors
Reducing data rank from 118 -> 118
Estimating covariance using EMPIRICAL
Done.
[ 0  0  0  0  0  0 -1  0 -1  0 -1 -1 -1  0 -1  0 -1  0 -1  0  0 -1 -1 -1
  0  0 -1  0  0 -1  0 -1 -1 -1 -1 -1 -1 -1 -1  0 -1  0 -1  0 -1 -1  0  0
  0  0  0  0  0  0  0 -1]
[ 0  0  0  0  0  0 -1  0 -1  0 -1 -1 -1  0 -1  0 -1 -1 -1  0  0  0  0 -1
 -1  0 -1 -1  0 -1  0 -1 -1 -1 -1 -1  0 -1 -1  0 -1  0 -1  0 -1 -1 -1  0
  0  0  0  0 -1  0  0 -1]


In [5]:
print( met.confusion_matrix(test_y, result) )
print( "Recall: ", met.recall_score(test_y, result, average=None) )
print( "Precision: ", met.precision_score(test_y, result, average=None) )
print( "F1: ", met.f1_score(test_y, result, average=None) )
print( "Acurracy: ", met.accuracy_score(test_y, result) )

tn, fp, fn, tp = met.confusion_matrix(test_y, result).ravel()
print(tn, fp, fn, tp)

print(met.classification_report(test_y, result))


[[23  5]
 [ 3 25]]
Recall:  [0.82142857 0.89285714]
Precision:  [0.88461538 0.83333333]
F1:  [0.85185185 0.86206897]
Acurracy:  0.8571428571428571
23 5 3 25
              precision    recall  f1-score   support

          -1       0.88      0.82      0.85        28
           0       0.83      0.89      0.86        28

    accuracy                           0.86        56
   macro avg       0.86      0.86      0.86        56
weighted avg       0.86      0.86      0.86        56

